# Training Digital Auditing

## Code Workshop: mastering the AI assistant

In [ ]:
# Initiate Google Drive connection
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import pandas as pd
import os


In [ ]:
# Read in your dataset with journal entries
gl_data = pd.read_excel("/content/drive/MyDrive/NBADataScience/Audit cases/DigiJazz_transactions.xlsx", sheet_name="GLData")

# Disable scientific notation, and enable thousand separator
pd.options.display.float_format = '{:,.2f}'.format

gl_data.head(5)

We hebben de volgende kolommen in onze dataset:

In [ ]:
gl_data.columns

### Maak een 'AmountNet' kolom

In [ ]:
# Set AmountNet to 1x the Amount for Debit transactions
gl_data.loc[gl_data['Debit/Credit'] == 'Debit', 'AmountNet'] = gl_data['Amount']
# Set AmountNet to 1x the Amount for Credit transactions
gl_data.loc[gl_data['Debit/Credit'] != 'Debit', 'AmountNet'] = gl_data['Amount'] * -1

### Q1. Maak een Trial Balance om aan te sluiten op het dossier:

In [ ]:
gl_data.groupby('Account')['AmountNet'].sum().reset_index()

In [ ]:
##

# Revenue analysis

### Exploration

GL/Journal matrix, analyzing the Journals posting on revenue:

In [ ]:
df_revenue = gl_data[gl_data['AccountType'] == 'Revenue']

# Create a GL/Journal matrix
df_revenue.pivot_table(index='Account', columns='JournalDesc', values='AmountNet', aggfunc='sum')

GL/Journal matrix, analyzing the GL accounts posting using the sales journal:

In [ ]:
df_revenue_journal = gl_data[gl_data['JournalType'] == 'Sales']

# Create a GL/Journal matrix
df_revenue_journal.pivot_table(index='Account', columns='JournalDesc', values='AmountNet', aggfunc='sum')

Plot of revenue over time:

In [ ]:
import matplotlib.pyplot as plt

data = df_revenue.groupby('Period')['Amount'].sum()
x = data.index
y = data.values

# Create a bar chart
plt.bar(x, y) # <---- changed from plot to bar! Easy as that.

# Add labels and a title
plt.xlabel('Period')
plt.ylabel('Revenue')
plt.title('Revenue per Period')
plt.xticks(rotation=45) # Rotate the x-axis labels

# Show the plot
plt.show()

### Reconcile Revenue to Sales Journal

In [ ]:
# Create a pivot table
df_revenue.pivot_table(index='User', columns='JournalDesc', values='AmountNet', aggfunc='sum')

### Risk analysis: understanding

In [ ]:
# Mean transaction amount
df_revenue['Amount'].mean()

In [ ]:
# Max transaction amount
df_revenue['Amount'].max()

In [ ]:
# Groupby, sum and make sure no scientific notation is used
df_revenue.groupby('User')['Amount'].sum().apply(lambda x: '%.0f' % x).reset_index()

### Margin analysis

In [ ]:
# Calculate margin per order (using the 'Source_id' column)
salmargin = gl_data[(gl_data['AccountType'] == 'Revenue') | (gl_data['AccountType'] == 'COGS')]
salmargin = salmargin.pivot_table(index='Source_id', columns='AccountType', values='AmountNet', aggfunc='sum').fillna(0).round(2)
salmargin['Margin'] = (-salmargin['Revenue'] - salmargin['COGS'])/(-salmargin['Revenue'])
salmargin['Margin%'] = (-salmargin['Revenue'] - salmargin['COGS'])/(-salmargin['Revenue'])*100

salmargin.head(5)

In [ ]:
salmargin_negative = salmargin[salmargin['Margin%'] < 0]
salmargin_negative.head(5)

In [ ]:
# We use the pyplot and seaborn libraries to visualize the data, refer to Cheatsheet Week 1a - Libraries and Exploring a dataset
import matplotlib.pyplot as plt
import seaborn as sns

# Analyzing transaction values for outliers
# Define thresholds based on typical transaction ranges
threshold_lower = 20 # Assumed min 20% margin based on knowledge from DigiJazz
threshold_upper = 50 # Assumed max 50% margin based on knowledge from DigiJazz

# Plot histogram for transaction values
plt.figure(figsize=(8, 6))
sns.histplot(salmargin['Margin%'], bins=20)
plt.axvline(threshold_lower, color='red', linestyle='--', label='Lower Threshold')
plt.axvline(threshold_upper, color='blue', linestyle='--', label='Upper Threshold')
plt.title('Distribution of Margins')
plt.xlabel('Margin %')
plt.legend()
plt.show()